In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt

In [2]:
# !pip install --upgrade pythainlp
# !pip install pyLDAvis

In [3]:
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings


In [4]:
df = pd.read_csv("sentimentMacbookAirM1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  81 non-null     object
dtypes: object(1)
memory usage: 776.0+ bytes


In [5]:
df = df.astype(str)
df

,message
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก
...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...


# Tokenize word

In [6]:
stop_word = list(pythainlp.corpus.thai_stopwords())
rm_word = [ ' ','    ','\n', '(', ')', '?', '+', 'ผม', ':', 'ตัว', 'เรื่อง', 'ส่วนตัว', 'คับ', '--', '//', 'แน่นอน', 'สอย', 'ซื้อ', 'เวลา', '' ]
screenword = stop_word + rm_word

def tokenize_n_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize( str(sentence), engine = 'newmm' )
    for i in words :
        if i not in screenword :
            merged = merged + ',' + i
    return merged[1:]

In [7]:
df['tokenized'] = df['message'].apply(lambda x: tokenize_n_space(x))
df

,message,tokenized
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...,"คอนเฟิร์ม,เสียง,Macbook,air,m,1,ลื่นไหล,แบต,อึ..."
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...,"สอบถาม,กะ,mac,air,m,1,ทำงาน,ตัดต่อ,กราฟิก,phot..."
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...,"M,1,ตัดต่อ,ระดับ,เทพ,ๆๆๆ,โปรแกรม,พัดลม"
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...,"เล่น,เกม,โน๊ตบุ๊ค,ดี,โลก,ราคา,28000,ทำได้,ประส..."
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก,"จุดเด่น,เสียง,ลำโพง,ดีมาก"
...,...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...,"ขออนุญาต,สอบถาม,นะคะ,macbook,pro,14,”,m,1,pro,..."
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...,"สอบถาม,apple,care,เครื่อง,icare,ต่างกัน, ,กลัว..."
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...,"ผลิตภัณฑ์,แอปเปิ้ล,ลังเล,บ้าน,apple,ประกัน,ตอน..."
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...,"ออกมา,ทุกปี,ดู,การใช้งาน,ประโยชน์,ที่จะ,คน,รอ,..."


# Creating Dictionary

In [8]:
doc = df['tokenized'].to_list()
texts = [ [i for i in dc.split(',')] for dc in doc ]

dictionary = gensim.corpora.Dictionary(texts)

print(dictionary.token2id.keys())

dict_keys(['1', 'Macbook', 'Performance', 'Windows', 'air', 'drop', 'm', 'คอนเฟิร์ม', 'ลืม', 'ลื่นไหล', 'สักพัก', 'อึด', 'เสียง', 'แบต', 'illustrator', 'mac', 'photoshop', 'กราฟิก', 'กะ', 'ตัดต่อ', 'ทำงาน', 'น้อง', 'สอบถาม', 'ไหว', 'M', 'พัดลม', 'ระดับ', 'เทพ', 'โปรแกรม', 'ๆๆๆ', '28000', 'การใช้งาน', 'ดี', 'ต่อให้', 'ทำ', 'ทำได้', 'ประสบการณ์', 'ราคา', 'ห่า', 'เกม', 'เล่น', 'โน๊ตบุ๊ค', 'โลก', 'จุดเด่น', 'ดีมาก', 'ลำโพง', '7', 'CTW', 'geniusbar', 'studio', 'ขาย', 'คนอื่น', 'คอม', 'จี', 'ดีกว่า', 'ดู', 'ที้ง', 'นิดนึง', 'บริการ', 'พนักงาน', 'ระบาย', 'รุ', 'ร้าน', 'สอง', 'สัก', 'สาขา', 'ห่วย', 'เข้าไป', 'เจอ', 'เนาะ', 'เนียส', 'แย่', 'แอปเปิ้ล', 'ใช้งาน', '2013', '2020', '2023', '34', '900', 'Air', 'คน', 'ค่ำ', 'งานเอกสาร', 'ตอน', 'ต้นปี', 'ประชุม', 'ประทับใจ', 'ปี', 'มีปัญหา', 'ยัน', 'รุ่น', 'หาย', 'เก่า', 'เครื่อง', 'เดิม', 'เปิดตัว', 'เลือก', 'แชร์', 'แต่เช้า', 'ไว', '/', '4', 'adobe', 'apple', 'exel', 'premia', 'word', 'การตัดต่อ', 'ขั้นพื้นฐาน', 'ความแตกต่าง', 'ค่', 'ง', 'จ่าย', 'ดร'

In [9]:
gensim_corpus = [ dictionary.doc2bow(tx, allow_update = True) for tx in texts ]
word_frq = [ [ (dictionary[ides], frq) for ides, frq in coup ] for coup in gensim_corpus ]

# Topic Modeling

In [10]:
num_topic = 30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_number = 1 # more number, more compute time spending

# Make index to word dictionary
temp = dictionary[0]
id2word = dictionary.id2token

%time
model = gensim.models.LdaModel( corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, alpha='auto', eta='auto'
                               , iterations=iterations, num_topics=num_topic, passes=passes, eval_every=eval_number )

Wall time: 0 ns


In [11]:
pyLDAvis.gensim.prepare( model, gensim_corpus, dictionary )

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.128593 -0.006871       1        1  10.295190
24    -0.072233 -0.157039       2        1   9.394821
22    -0.070455 -0.165387       3        1   8.892813
21    -0.107755  0.137801       4        1   7.819040
1     -0.042202  0.141535       5        1   7.250903
29    -0.110274  0.015543       6        1   5.396550
14    -0.004402  0.081699       7        1   5.390920
5      0.061363 -0.036578       8        1   4.099566
8     -0.072913  0.053846       9        1   4.070501
17    -0.083783 -0.040399      10        1   3.993919
4     -0.134053 -0.038860      11        1   3.661676
23     0.120394  0.067559      12        1   3.416837
11     0.088936 -0.036281      13        1   3.058516
25    -0.031861  0.041621      14        1   2.994464
16     0.176630 -0.025773      15        1   2.623299
13    -0.007378 -0.015949      16        1   2.386693
7     -0.009603  0.052762      17        1   2.335077
9      0.020867 -0.053301      18        1   2.175496
19    -0.018255  0.013236      19        1   1.995230
28     0.010680 -0.037353      20        1   1.833092
12     0.057854  0.004863      21        1   1.442746
3      0.056565 -0.030160      22        1   1.201116
20     0.012874  0.030041      23        1   1.123953
27    -0.000633  0.001693      24        1   1.072893
10     0.022140 -0.016516      25        1   0.781231
26     0.073063  0.015539      26        1   0.569845
6      0.031852  0.007839      27        1   0.517318
0      0.053674 -0.001683      28        1   0.069693
2      0.053740 -0.001707      29        1   0.068578
18     0.053760 -0.001720      30        1   0.068022, topic_info=        Term       Freq      Total Category  logprob  loglift
169           13.000000  13.000000  Default  30.0000  30.0000
4        air  19.000000  19.000000  Default  29.0000  29.0000
28   โปรแกรม  11.000000  11.000000  Default  28.0000  28.0000
0          1  34.000000  34.000000  Default  27.0000  27.0000
24         M  19.000000  19.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25     พัดลม   0.001477   7.719076  Topic30  -6.7731  -1.2682
26     ระดับ   0.001477   3.759365  Topic30  -6.7731  -0.5488
27       เทพ   0.001477   1.260193  Topic30  -6.7731   0.5442
28   โปรแกรม   0.001477  11.128850  Topic30  -6.7731  -1.6341
29       ๆๆๆ   0.001477   1.260193  Topic30  -6.7731   0.5442

[1398 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
169       7  0.148974       
169       9  0.670383       
169      14  0.223461       
818       3  0.880982       
224      12  0.796753      %
...     ...       ...    ...
266      10  0.776870  ไอโฟน
825       3  0.880982     ๆ?
29       11  0.793529    ๆๆๆ
833       4  0.717054      ”
844       6  0.748805    🥹😭 

[1240 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 25, 23, 22, 2, 30, 15, 6, 9, 18, 5, 24, 12, 26, 17, 14, 8, 10, 20, 29, 13, 4, 21, 28, 11, 27, 7, 1, 3, 19])

- Predict topic

In [12]:
model.show_topic(1)

[('ไฟล์', 0.01801895),
 ('เครื่อง', 0.018017627),
 ('1', 0.012851783),
 ('ใช้งาน', 0.01208207),
 ('K', 0.012078577),
 ('ระดับ', 0.012078577),
 ('8', 0.012078577),
 ('GB', 0.012078577),
 ('Macbook', 0.012078574),
 ('air', 0.012078574)]

In [13]:
df['topics'] = df['tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [14]:
df

,message,tokenized,topics,score
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...,"คอนเฟิร์ม,เสียง,Macbook,air,m,1,ลื่นไหล,แบต,อึ...",1,0.959417
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...,"สอบถาม,กะ,mac,air,m,1,ทำงาน,ตัดต่อ,กราฟิก,phot...",27,0.955853
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...,"M,1,ตัดต่อ,ระดับ,เทพ,ๆๆๆ,โปรแกรม,พัดลม",4,0.930959
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...,"เล่น,เกม,โน๊ตบุ๊ค,ดี,โลก,ราคา,28000,ทำได้,ประส...",9,0.967685
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก,"จุดเด่น,เสียง,ลำโพง,ดีมาก",11,0.870034
...,...,...,...,...
76,ขออนุญาตสอบถามหน่อยนะคะ คือเราซื้อ macbook pro...,"ขออนุญาต,สอบถาม,นะคะ,macbook,pro,14,”,m,1,pro,...",21,0.966231
77,อยากสอบถามทุกคนหน่อยค่ะ ว่า apple care+ ควรซื้...,"สอบถาม,apple,care,เครื่อง,icare,ต่างกัน, ,กลัว...",29,0.980565
78,ตั้งแต่เราซื้อผลิตภัณฑ์แอปเปิ้ลมาลังเลตลอดว่าจ...,"ผลิตภัณฑ์,แอปเปิ้ล,ลังเล,บ้าน,apple,ประกัน,ตอน...",1,0.986994
79,มันก็ออกมาเรื่อยทุกปีแหละครับ ดูการใช้งานและปร...,"ออกมา,ทุกปี,ดู,การใช้งาน,ประโยชน์,ที่จะ,คน,รอ,...",24,0.981079


In [15]:
# !pip install tensorflow_text
# !pip install umap-learn

# Document Clustering

In [18]:
import re
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import umap

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

from pythainlp.corpus.common import thai_words
from pythainlp.util import Trie
import collections


In [19]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'

model = hub.load(module_url)

df_k = pd.read_csv("sentimentMacbookAirM1.csv")
df_k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  81 non-null     object
dtypes: object(1)
memory usage: 776.0+ bytes


In [20]:
embed_cm = model(df['message'].values).numpy()
embed_cm

array([[-0.03456426,  0.05708109,  0.01469755, ...,  0.01186927,
        -0.01357166,  0.09678852],
       [-0.01054443,  0.03593316,  0.03521626, ..., -0.05023287,
        -0.02311422,  0.05664483],
       [ 0.04410389,  0.02387322,  0.01173277, ...,  0.00014756,
         0.08564217,  0.02562594],
       ...,
       [-0.0236195 ,  0.02566138, -0.0348949 , ..., -0.05413022,
         0.04013607,  0.03504496],
       [ 0.08128867,  0.01058034, -0.0175554 , ..., -0.07995252,
         0.08278595,  0.06422471],
       [-0.04074633,  0.07052976,  0.02234775, ...,  0.03710676,
         0.05302522, -0.03792713]], dtype=float32)

In [21]:
reducer = umap.UMAP(random_state=42, n_components=50)
umap_embed = reducer.fit_transform(embed_cm)

In [24]:
wcss = []
max_k = 10

for i in range(1,max_k):
    kmean = KMeans(i, n_init='auto')
    kmean.fit(umap_embed)
    
    wcss.append(kmean.inertia_)

n_cluster = range(1,max_k)
plt.plot(n_cluster, wcss)
plt.title('K-Elbow')
plt.xlabel('Number of Cluster')
plt.ylabel('WCSS')

TypeError: KMeans.__init__() got an unexpected keyword argument 'OMP_NUM_THREADS'